In [1]:
import setup_django
setup_django.init()

In [2]:
import pandas as pd
from django.conf import settings

ratings_path = settings.DATA_DIR / "ratings_small.csv"
ratings_path.exists()

True

In [4]:
df = pd.read_csv(ratings_path)
df.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
from users.models import User

current_users = User.objects.all().values_list('id', flat=True)
rating_users = df['userId'].tolist()
missing_user_ids = set(rating_users) - set(current_users)
print(missing_user_ids)

set()


In [7]:
for uid in missing_user_ids:
    User.objects.create(
        id=uid,
        username = f"missing-user-{uid}"
    )

In [9]:
import math
from decimal import Decimal
df['value'] = df['rating'].apply(lambda x: math.ceil(Decimal(x)))
df.head(5)

,userId,movieId,rating,timestamp,value
0,1,31,2.5,1260759144,3
1,1,1029,3.0,1260759179,3
2,1,1061,3.0,1260759182,3
3,1,1129,2.0,1260759185,2
4,1,1172,4.0,1260759205,4


In [10]:
df['user_id'] = df['userId']
df['object_id'] = df['movieId']
df.head()

,userId,movieId,rating,timestamp,value,user_id,object_id
0,1,31,2.5,1260759144,3,1,31
1,1,1029,3.0,1260759179,3,1,1029
2,1,1061,3.0,1260759182,3,1,1061
3,1,1129,2.0,1260759185,2,1,1129
4,1,1172,4.0,1260759205,4,1,1172


In [12]:
cols = ['user_id', 'value', 'object_id']
transformed_df = df.copy()[cols]
transformed_df.head()

,user_id,value,object_id
0,1,3,31
1,1,3,1029
2,1,3,1061
3,1,2,1129
4,1,4,1172


In [13]:
rating_records = transformed_df.to_dict('records')
from ratings.models import Rating
qs = Rating.objects.all()
qs.delete()

(2256994, {'ratings.Rating': 2256994})

In [14]:
from django.contrib.contenttypes.models import ContentType
ctype = ContentType.objects.get(app_label='movies', model='movie')


In [17]:
new_rating = []
for r in rating_records:
    r['content_type'] = ctype
    new_rating.append(
        Rating(**r)
    )
Rating.objects.bulk_create(new_rating,ignore_conflicts=True, batch_size=1000)

[<Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: 

In [18]:
from ratings.tasks import task_update_movie_ratings

task_update_movie_ratings()